# M2 Bending Mode Test

This Jupyter notebook is to run the bending mode test of M2.
Each axial actuator will be issued with an additional force according to specific bending mode (there are 69 in total).
The target is to understand the mirror supporting system can hold the specific shape based on bending mode or not.

IMPORTANT NOTE: the bending mode dataset starts from index 0.

## Import Modules

In [ ]:
import asyncio
import yaml
import numpy as np
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK
from lsst.ts.ofc.utils import get_config_dir as get_config_dir_ofc

## Declaration of User-defined Functions

In [ ]:
def get_bending_mode():
    """Get the bending mode.

    Returns
    -------
    bending_mode : `numpy.ndarray`
        Bending mode.
    """
    bending_mode_file = get_config_dir_ofc() / "M2" / "M2_1um_72_force.yaml"
    with open(bending_mode_file, "r") as yaml_file:
        bending_mode = np.array(yaml.safe_load(yaml_file))

    
    return bending_mode

In [ ]:
def get_bending_mode_forces(bending_mode, idx_bending_mode, amplitude):
    """Plot the forces of bending mode.
    
    Parameters
    ----------
    bending_mode : `numpy.ndarray`
        Bending mode data.
    idx_bending_mode : `int`
        Index of bending mode (1-20).
    amplitude : `float`
        Amplitude of bending mode in um.
        
    Returns
    -------
    `numpy.ndarray`
        Actuator forces in Newton.
    """

    # Column 0-2 are the actuator ID, x and y position of actuator
    return amplitude * bending_mode[:, 2+idx_bending_mode]

In [ ]:
async def apply_force_cycle_axial(csc, forces, time_forces=5, time_reset=5):
    """Apply the force cycle to axial actuators. The cycle will be positive force, clear,
    negative force, clear.

    Parameters
    ----------
    csc : `lsst.ts.salobj.remote.Remote`
        Remote object of the M2 CSC.
    forces : `numpy.ndarray`
        Axial forces to apply (Newton).
    time_forces : `float`, optional
        Time to apply the forces in second. (the default is 5.0) 
    time_reset : `float`, optional
        Time to reset the forces in second. (the default is 5.0)
    """
    
    # Do the positive direction first
    print(f"Apply the force: {forces} N.")
    await csc.cmd_applyForces.set_start(axial=forces.tolist())
    await asyncio.sleep(time_forces)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(time_reset)

    # Do the Negative direction
    forces_negative = -forces

    print(f"Apply the force: {forces_negative} N.")
    await csc.cmd_applyForces.set_start(axial=forces_negative.tolist())
    await asyncio.sleep(time_forces)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(time_reset)

## Prepare the M2 CSC and put to Enabled state

In [ ]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [1]:
# get status

state = m2.evt_summaryState.get()
print(state)

NameError: name 'm2' is not defined

In [ ]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=60)

In [ ]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=200)

In [ ]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

## Apply one Bending Modes to fine tune the amplitude/scaling factor

The fine tuning of the bending mode scaling factor is done for groups of 10 bending modes at the time. This step can be skipped if the scaling factors are already defined.

In [ ]:
bending_mode = get_bending_mode()

In [ ]:
# bending modes vector indexes
decina = 0 #Select the tens-order of the bending mode, from 0 to 6  
idx = 0 #Select the bending mode inside the tens-order, from 0 to 9

# genereate a vector of 10 elements to be populated with the ascending bending mode order 
idx_bending_mode = [0] * 10
for idx in range(10):
    idx_bending_mode[idx] = idx + decina*10
    
print(idx_bending_mode)
# bending mode scaling factors, to be fine tuned, applied force shall be around 10 N to get 500 nm RMS surface deformation
amplitude = np.array([1, 0.1, 0.1, 0.05, 0.025, 0.01, 0.01])


In [ ]:
# bending mode number from 0 to 9

print(f"Bending mode is {idx_bending_mode[idx]} and amplitude is {amplitude[decina]}")
forces = get_bending_mode_forces(bending_mode, idx_bending_mode[idx]+1, amplitude[0])

# apply single bending mode
await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)

## Looping over multiple bending modes 
Loop over multiple bending modes with positive and negative signs at groups of 10 bending modes. 

In [ ]:
bending_mode = get_bending_mode()

In [ ]:
# array definition

# bending modes scaling factors
amplitude = np.array([1, 0.1, 0.1, 0.05, 0.025, 0.01, 0.01])

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")


# bending modes from 1 to 10

for idx in range(0,10):
    print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[0]}")
    forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[0])
    # apply single +/- bending mode 
    await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 11 to 20

for idx in range(10,20):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[1]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[1])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 21 to 30

for idx in range(20,30):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[2]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[2])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 31 to 40

for idx in range(30,40):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[3]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[3])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 41 to 50

for idx in range(40,50):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[4]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[4])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 51 to 60

for idx in range(50,60):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[5]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[5])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending modes from 61 to 69

for idx in range(60,68):
        print(f"Status of advancement: Bending mode {idx+1}, mode amplitude {amplitude[6]}")
        forces = get_bending_mode_forces(bending_mode, idx+1, amplitude[6])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

## Bending modes repeatability
Apply for 3 times the same bending mode with positive and negative sign

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

# bending mode 15 repeatability x3 times

for repetition in range(0,3):
        print(f"Status of advancement: Bending mode {15}, mode amplitude {amplitude[1]}")
        forces = get_bending_mode_forces(bending_mode, 15, amplitude[1])
        # apply single +/- bending mode 
        await apply_force_cycle_axial(m2, forces, time_forces=5, time_reset=5)
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")